In [3]:
import pandas as pd

In [4]:
data_path = '/data_volume/memento/lupus/ldsc/'

In [5]:
cts =['T8', 'T4', 'NK', 'ncM', 'cM', 'B']


### Read the gene loc file

In [6]:
gene_loc = pd.read_table(data_path + 'resources/gene_locs.txt')

In [7]:
# Add 100k window
gene_loc['start'] = (gene_loc['start'] - 100000).clip(lower=0)
gene_loc['end'] = gene_loc['end'] + 100000

### Get egenes and make bedfiles for memento

In [20]:
for pop in ['asian', 'eur']:
    for ct in cts:
        print(pop, ct)
        result = pd.read_csv('/data_volume/memento/lupus/full_analysis/memento/100kb/{}_{}.csv'.format(pop, ct))
        result.query('FDR < 0.05')[['gene']].drop_duplicates().to_csv(data_path + 'egenes/memento/{}_{}.txt'.format(pop, ct),index=False, header=False, sep='\t')        

asian T4
asian cM
asian ncM
asian T8
asian B
asian NK
eur T4
eur cM
eur ncM
eur T8
eur B
eur NK


In [24]:
for pop in ['eur', 'asian']:
    for ct in cts:
        egenes = pd.read_table(data_path + 'egenes/memento/{}_{}.txt'.format(pop, ct), header=None).iloc[:, 0].tolist()
        gene_loc.query('name in @egenes')[['chr', 'start', 'end', 'name']].to_csv(data_path + 'bedfiles/memento/{}_{}.bed'.format(pop, ct), index=False, header=False, sep='\t')

### Get egenes and make bedfiles for matrix eqtl

In [11]:
for pop in ['asian', 'eur']:
    for ct in cts:
        print(pop, ct)
        result = pd.read_table('/data_volume/memento/lupus/full_analysis/mateqtl/outputs/{}_{}_all_hg19.csv'.format(pop, ct))
        result.query('FDR < 0.05')[['gene']].drop_duplicates().to_csv(data_path + 'egenes/mateqtl/{}_{}.txt'.format(pop, ct),index=False, header=False, sep='\t')        

asian T8
asian T4
asian NK
asian ncM
asian cM
asian B
eur T8
eur T4
eur NK
eur ncM
eur cM
eur B


In [15]:
cts =['T8', 'T4', 'NK', 'ncM', 'cM', 'B']
for pop in ['eur', 'asian']:
    for ct in cts:
        egenes = pd.read_table(data_path + 'egenes/mateqtl/{}_{}.txt'.format(pop, ct), header=None).iloc[:, 0].tolist()
        gene_loc.query('name in @egenes')[['chr', 'start', 'end', 'name']].to_csv(data_path + 'bedfiles/mateqtl/{}_{}.bed'.format(pop, ct), index=False, header=False, sep='\t')

### Get "egenes" and make bedfile for all expressed genes

In [16]:
all_genes = set()
for pop in ['asian', 'eur']:
    for ct in cts:
        result = pd.read_csv('/data_volume/memento/lupus/full_analysis/memento/100kb/{}_{}.csv'.format(pop, ct))
        all_genes |= set(result['gene'])
gene_loc.query('name in @all_genes')[['chr', 'start', 'end', 'name']].to_csv(data_path + 'bedfiles/pbmc.bed'.format(pop, ct), index=False, header=False, sep='\t')